### Gerekli Kütüphanelerin Yüklenmesi

In [1]:
import cv2
import numpy as np
import tensorflow as tf

# Modeli yükle
model = tf.keras.models.load_model("model.keras")

### Etiketlerin Tanımlanması

In [2]:
class_labels = [ "glass", "paper", "cardboard", "plastic", "metal", "trash" ] 

### Gerçek Zamanlı Nesne Tanıma

In [11]:
# Kamerayı başlat
cap = cv2.VideoCapture(0)

# Tespit için güven sınırı
CONFIDENCE_THRESHOLD = 0.8

detected_class_name = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Kameradan görüntü alınamadı.")
        break

    # OpenCV ile görüntüyü yeniden boyutlandır ve işleme hazır hale getir
    input_image = cv2.resize(frame, (224, 224))  # MobilNetV2 için varsayılan boyut
    input_image = np.expand_dims(input_image, axis=0)  # Batch boyutunu ekle
    input_image = input_image / 255.0  # Normalize et

    # Model ile tahmin yap
    predictions = model.predict(input_image)
    confidence = np.max(predictions)
    predicted_class_index = np.argmax(predictions)  # En yüksek tahmin indeksini al
    predicted_class_name = class_labels[predicted_class_index]  # İndeksi sınıf adına dönüştür

    # Tespit edilen sınıfı ve güven puanını göster
    cv2.putText(frame, f"Class: {predicted_class_name}, Confidence: {confidence:.2f}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Real-time Detection", frame)

    # Güven sınırını geçen sınıf bulunursa işlemi durdur
    if confidence > CONFIDENCE_THRESHOLD:
        detected_class_name = predicted_class_name
        print(f"Tespit edilen sınıf: {detected_class_name}, Güven: {confidence:.2f}")
        break

    # Çıkmak için 'q' tuşuna bas
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Kamerayı serbest bırak ve tüm pencereleri kapat
cap.release()
cv2.destroyAllWindows()

# Tespit edilen sınıfı döndür
if detected_class_name is not None:
    print(f"Kamera kapatıldı. Tespit edilen sınıf: {detected_class_name}")
else:
    print("Hiçbir sınıf belirlenen güven seviyesine ulaşmadı.")

1/1 [==============================] - 0s 89ms/step
Tespit edilen sınıf: cardboard, Güven: 0.85
Kamera kapatıldı. Tespit edilen sınıf: cardboard
